<a href="https://colab.research.google.com/github/Eminent01/AMMI-WORK/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import seaborn as sns
import cv2
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, random_split
from PIL import Image

import torch.nn.functional as F
from torchvision.models import resnet18
from torch.utils.data import DataLoader,random_split
import torch.optim as optim

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
path= "/content/drive/MyDrive/Cat_Dog_Images/Images"

In [ ]:
# metadata.to_csv("/content/drive/MyDrive/Cat_Dog_Images/metadata_ok.csv", index= False)

In [ ]:
metadata= "/content/drive/MyDrive/Cat_Dog_Images/metadata_ok.csv"

In [ ]:
# pd.read_csv(metadata, header= 1)

In [ ]:
class CustomDataSet(Dataset):
    def __init__(self, metadata, dir_root,transform= None):
        self.dir_root= dir_root
        self.annotations= pd.read_csv(metadata, header= 1)
        self.transform= transform
        
        
    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        
        fileName= self.annotations.iloc[idx, 0]
        im= os.path.join(self.dir_root, fileName)
        img = Image.open(im).convert("RGB")
        
        if self.transform:
            img = self.transform(img)
        
        classCategory= self.annotations.iloc[idx, 1]
        return img, im, classCategory

In [ ]:
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.RandomResizedCrop(224),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])


In [ ]:
data= CustomDataSet(metadata, path, transform= transform)

In [ ]:
data

In [ ]:
len(data)

In [ ]:
Image.open(data[10][1])

In [ ]:
def img_display(img):
    # img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    npimg = np.transpose(npimg, (1, 2, 0))
    return npimg

In [ ]:
batch_size= 16
train_size= 0.8
train_size= int(train_size*len(data))
val_size = len(data) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(data, [train_size, val_size])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
data[0][0].shape

In [ ]:
# for i, _, j in train_loader:
#   print(i)

In [ ]:
dataiter = iter(train_loader)
images, img, labels= dataiter.next()
bird_type = {0: 'cat', 1: 'dog'}
# Viewing data examples used for training
fig, axis = plt.subplots(2, 2, figsize=(15, 10))
for i, ax in enumerate(axis.flat):
    with torch.no_grad():
        image, label= images[i], labels[i]
        ax.imshow(img_display(image)) # add image
        
        ax.set(title = f"{bird_type[label.item()]}")

In [ ]:
dataiter = iter(val_loader)
images, img, labels= dataiter.next()
bird_type = {0: 'cat', 1: 'dog'}
# Viewing data examples used for training
fig, axis = plt.subplots(2, 2, figsize=(15, 10))
for i, ax in enumerate(axis.flat):
    with torch.no_grad():
        image, label= images[i], labels[i]
        ax.imshow(img_display(image)) # add image
        
        ax.set(title = f"{bird_type[label.item()]}")

In [ ]:
# train_loader

In [ ]:
model= torchvision.models.resnet50(pretrained= True)
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(2048, 1, bias = True),
    nn.Sigmoid()
)

# model.to(device)

In [ ]:
criterion= nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, weight_decay=1.0)
num_epochs= 30

In [ ]:


def train(model, criterion, train_loader, val_loader, optimizer, num_epochs, device):
    """Simple training loop for a PyTorch model.""" 

    val_loss = []
    val_acc = []
    train_loss = []
    train_acc = []
    val_loss_min = np.Inf
    total_step = len(train_loader)
    
    # Make sure model is in training mode.
    model.train()
    
    # Move model to the device (CPU or GPU).
    model.to(device)
    
    # Exponential moving average of the loss.
    ema_loss = None
    
    # Loop over epochs.
    # for epoch in range(num_epochs):
    for epoch in tqdm(range(num_epochs)):
      running_loss = 0.0
      correct = 0
      total = 0
        
      # Loop over data.
      for data, _, target in train_loader:
            
          # Forward pass.
          output =model(data.to(device))
          loss = criterion(output.to(device).squeeze(), target.to(device).float())
          # loss.requires_grad = True
          # print(loss)
          # Backward pass.
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          running_loss +=loss.item()
          pred = output.argmax(dim=1, keepdim=True)
          correct += pred.cpu().eq(target.view_as(pred)).sum().item()
          total += target.size(0)
          
      print('Train Epoch: {} \tLoss: {:.6f}'.format(
            epoch, running_loss/total_step),
      )
      
      train_acc.append(100 * correct/total)
      train_loss.append(running_loss/total_step)



      # Validation
      batch_loss = 0
      total_t = 0
      correct_t = 0
      with torch.no_grad():
          model.eval()
          for data_t,_, target_t in val_loader:
              data_t,target_t = data_t.to(device),target_t.to(device)
              outputs_t = model(data_t)
              loss_t = criterion(outputs_t.squeeze(),target_t.float())
              batch_loss+=loss.item()
              _,pred_t = torch.max(outputs_t,dim=1)
              correct_t += torch.sum(pred_t==target_t).item()
              total_t += target_t.size(0)
          val_acc.append(100 * correct_t/total_t)
          val_loss.append(batch_loss/len(val_loader))
          network_achieve = batch_loss < val_loss_min
        

          if network_achieve:
              val_loss_min = batch_loss
              torch.save(model.state_dict(), '/content/model_ok.ckpt')

              print('The best model is detected')


    # torch.save(model.state_dict(), 'model.pth')
    percent = 100. * correct / len(train_loader.dataset)
    return model, percent, val_loss, val_acc, train_loss, train_acc

In [ ]:
model_trained, percent, val_loss, val_acc, train_loss, train_acc= train(model, criterion, train_loader, val_loader, optimizer, num_epochs, device)

In [ ]:
plt.title("Training results: Loss")
plt.plot(val_loss,label='val_loss')
plt.plot(train_loss, label="train_loss")
plt.legend()

In [ ]:
plt.title("Training results: Acc")
plt.plot(val_acc,label='val_acc')
plt.plot(train_acc, label="train_acc")
plt.legend()

In [ ]:
val_acc

In [ ]:
train_acc

In [ ]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = torch.nn.Linear(9408, 1, bias=True)
        self.m = nn.Sigmoid()
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  
        # print(out.shape) 
        out = self.fc(out)
        out= self.m(out)
        return out

# instantiate CNN model
model = CNN().to(device)

In [ ]:
model_trained, percent, val_loss, val_acc, train_loss, train_acc= train(model, criterion, train_loader, optimizer, num_epochs)

In [ ]:
plt.title("Training results: Loss")
plt.plot(val_loss,label='val_loss')
plt.plot(train_loss, label="train_loss")
plt.legend()

In [ ]:
#plt.title("Training results: Loss")
plt.plot(val_loss,label='val_loss')
plt.plot(train_loss, label="train_loss")
plt.legend()

In [ ]:
## Add a function form predict in local (predict.py)

In [ ]:
model_train, train_loss= train(model, criterion, train_loader, optimizer, num_epochs=10)

In [ ]:
_, val_loss= test(model_train, val_loader)

In [ ]:
val_loss

In [ ]:
plt.title("Training results")
plt.plot(val_loss,label='val_loss')
plt.plot(train_loss, label="train_loss")
plt.legend()

In [ ]:
# Prediction
def image_transform(imagepath):
    test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
    image = Image.open(imagepath)
    imagetensor = test_transforms(image)
    return imagetensor


def predict(imagepath, verbose=False):
    if not verbose:
        warnings.filterwarnings('ignore')
    model_path = './model/model_ok.pth'
    try:
        checks_if_model_is_loaded = type(model)
    except:
        model = models.resnet50()
    model.eval()
    #summary(model, input_size=(3,244,244))
    if verbose:
        print("Model Loaded..")
    image = image_transform(imagepath)
    image1 = image[None,:,:,:]
    # ps= torch.exp(model(image1))
    ps= model(image1)
    topconf, topclass = ps.topk(1, dim=1)
    # print(ps)
    if topclass.item() == 1:
        return {'class':'dog','confidence':str(topconf.item())}
    else:
        return {'class':'cat','confidence':str(topconf.item())}



image_path= "Image_path"
# print(predict('data/Images/image_name'))
print(Image.open(image_path))
print(predict(image_path))